<a href="https://colab.research.google.com/github/vence-andersen/M5-Forecasting-Accuracy/blob/main/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import files
# files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"grahamfernando","key":"cffa546ccec7808cfc8cec9944ed384a"}'}

In [2]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [3]:
# ! kaggle competitions download -c m5-forecasting-accuracy

 32% 5.00M/15.8M [00:00<00:01, 8.75MB/s]
100% 15.8M/15.8M [00:00<00:00, 25.0MB/s]
  0% 0.00/101k [00:00<?, ?B/s]
100% 101k/101k [00:00<00:00, 89.6MB/s]
  0% 0.00/163k [00:00<?, ?B/s]
100% 163k/163k [00:00<00:00, 152MB/s]
 58% 9.00M/15.5M [00:00<00:00, 14.5MB/s]
100% 15.5M/15.5M [00:00<00:00, 24.5MB/s]
 35% 5.00M/14.2M [00:01<00:02, 4.71MB/s]
100% 14.2M/14.2M [00:01<00:00, 12.8MB/s]


In [4]:
# !unzip sales_train_evaluation.csv.zip
# !unzip sales_train_validation.csv.zip
# !unzip sample_submission.csv.zip
# !unzip sell_prices.csv.zip

Archive:  sales_train_evaluation.csv.zip
  inflating: sales_train_evaluation.csv  
Archive:  sales_train_validation.csv.zip
  inflating: sales_train_validation.csv  
Archive:  sample_submission.csv.zip
  inflating: sample_submission.csv   
Archive:  sell_prices.csv.zip
  inflating: sell_prices.csv         


In [7]:
# ! pip install downcast

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from downcast import reduce

In [6]:
# Using Pandas to reasd the files.

sales = pd.read_csv("sales_train_evaluation.csv")
sell_price = pd.read_csv("sell_prices.csv")
cal = pd.read_csv("calendar.csv")

In [ ]:
# #https://www.kaggle.com/anshuls235/time-series-forecasting-eda-fe-modelling/notebook#2.-Downcasting

# def downcast(df):

#     cols = df.dtypes.index.tolist()
#     types = df.dtypes.values.tolist()

#     for i,t in enumerate(types):

#         if 'int' in str(t):
#             if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
#                 df[cols[i]] = df[cols[i]].astype(np.int8)
#             elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
#                 df[cols[i]] = df[cols[i]].astype(np.int16)
#             elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
#                 df[cols[i]] = df[cols[i]].astype(np.int32)
#             else:
#                 df[cols[i]] = df[cols[i]].astype(np.int64)

#         elif 'float' in str(t):
#             if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
#                 df[cols[i]] = df[cols[i]].astype(np.float16)
#             elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
#                 df[cols[i]] = df[cols[i]].astype(np.float32)
#             else:
#                 df[cols[i]] = df[cols[i]].astype(np.float64)

#         elif t == np.object:
#             if cols[i] == 'date':
#                 df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
#             else:
#                 df[cols[i]] = df[cols[i]].astype('category')

#     return df  

In [9]:
# To reduce the memory usage, we'll be using reduce throughout the code

sales = reduce(sales)
sell_price = reduce(sell_price)
cal = reduce(cal)

In [12]:
# We are creating new features required for the prediction for days from 1942 till 1969

for day in range(1942,1942+28):
  sales['d_' + str(day)] = np.int32(0)

In [13]:
# We are transforming our Time Series problem to Supervised Machine Learning Problem

data = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
          var_name='day', value_name='demand').dropna()
print("The new shape is ",data.shape)
data.head(2)

The new shape is  (60034810, 8)


,id,item_id,dept_id,cat_id,store_id,state_id,day,demand
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [14]:
# We are then merging the all the csv files together

data = data.merge(cal, left_on='day', right_on='d')

data = data.merge(sell_price,on=['store_id','item_id', 'wm_yr_wk'], how='left')

In [15]:
data['sell_price'].fillna(data.groupby('id')['sell_price'].transform('mean'), inplace=True)

In [16]:
# we are stripping the 'd_' from day column to make it an integer feature
data['day'] = data['day'].apply(lambda x: x.split('_')[1]).astype(np.int16)

#since weekday's are represented as wday with numbers and d is a duplicate column.
data.drop(['d','weekday','date'], axis=1, inplace=True) 

In [17]:
#  We are doing a categorical encoding by replacing the categories with the memory number every category has.
# This method is just a lazy way of doing things as cat codes are produced only features of types categories.
# Hence if a feature which is not a category is passed in, it throws AttributeError and thus the excep condition excutes
# And the error is handled

for i in data.columns:
    try:
        data[i] = data[i].cat.codes
    except AttributeError:
        pass

In [18]:
# Adding lag shift features as those are good time series feature engineering steps.

lags = [28,30,35,42,49,56,63,70]
for lag in tqdm(lags):
    data["lag_" + str(lag)] = data.groupby("id")["demand"].shift(lag).astype(np.float16)

100%|██████████| 6/6 [00:17<00:00,  2.91s/it]


In [ ]:
# # Adding rolling window feature with mean value

# means = [5,7,28,56]
# for mns in tqdm(means):
#     data['rolling_median_'+str(mns)] = data.groupby(['id'])['demand'].transform(lambda x: x.rolling(mns).median())

In [19]:
data = data[data['day']>1000]
data.head(2)

,id,item_id,dept_id,cat_id,store_id,state_id,day,demand,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_28,lag_30,lag_35,lag_42,lag_49,lag_56
30490000,14370,1437,3,1,0,0,1001,2,11339,7,10,2013,-1,-1,-1,-1,0,0,0,8.257812,1.0,0.0,2.0,1.0,0.0,1.0
30490001,14380,1438,3,1,0,0,1001,0,11339,7,10,2013,-1,-1,-1,-1,0,0,0,3.970703,2.0,0.0,0.0,0.0,1.0,1.0


In [20]:
data.isna().sum().sum()

0

In [ ]:
# data.to_pickle('new.pkl')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !cp new.pkl /content/drive/MyDrive

In [ ]:
# data = pd.read_pickle('/content/drive/MyDrive/new.pkl')
# data.head(2)

In [21]:
# Now we are splitting our data into, Train, Test, Cross Validate.
# Being a time series model, we are splitting the data based on time.

# Records till day 1914 will be used for training the model.
X_train = data[data['day']<1914]

# Records between days 1914 and 1941 will be used for Validation
X_val = data[(data['day']>=1914) & (data['day']<=1941)]

# Records after day 1942 will used for final test
X_test = data[(data['day']>=1942)]

y_train = X_train['demand']
y_val = X_val['demand']

# We are drpping the features which are not required.
X_train.drop(['demand'],axis = 1,inplace = True)
X_val.drop(['demand'],axis = 1,inplace = True)
X_test.drop(['demand'],axis = 1,inplace = True)

print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape)

(27837370, 25) (27837370,)
(853720, 25) (853720,)
(853720, 25)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
learning_rate = 0.071
num_leaves = 123
min_data_in_leaf = 147

In [23]:
#fit the model on best parameters

m_lgb = LGBMRegressor( learning_rate=learning_rate ,
                           num_leaves=num_leaves ,
                           min_data_in_leaf=min_data_in_leaf)

m_lgb.fit(X_train, y_train)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.071, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001,
              min_data_in_leaf=147, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_leaves=123, objective=None, random_state=None,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [24]:
pred_val_array  = m_lgb.predict(X_val)
pred_test_array  = m_lgb.predict(X_test)

pred_val_array = np.reshape(pred_val_array, (-1, 28),order = 'F')
pred_test_array = np.reshape(pred_test_array, (-1, 28),order = 'F')

In [25]:
sub_df = pd.read_csv("sample_submission.csv")
sub_1 = sub_df.iloc[:30490,:]
sub_2 = sub_df.iloc[30490:,:]
f_cols = sub_df.columns[1:]

for i in range(len(f_cols)):
    sub_1[f_cols[i]] = pred_val_array[:,i]
    sub_2[f_cols[i]] = pred_test_array[:,i]

sub = pd.concat([sub_1,sub_2])
sub

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.528932,0.799976,0.682221,0.458770,1.156034,0.447171,0.850492,0.385679,1.150229,0.838759,1.457656,1.303053,0.970518,1.052068,0.421365,1.235182,0.740752,0.774385,0.765635,1.107423,1.133757,0.788390,0.935823,0.825556,1.266658,0.503187,1.478406,0.737714
1,HOBBIES_1_002_CA_1_validation,0.489853,0.299541,0.304200,0.493065,0.319192,0.489625,0.469219,0.384393,0.207542,0.211428,0.379079,0.324443,0.472846,0.387394,0.302558,0.209696,0.200079,0.300456,0.318369,0.392088,0.397126,0.286807,0.194347,0.323625,0.260055,0.401578,0.359576,0.359576
2,HOBBIES_1_003_CA_1_validation,0.399944,0.435021,0.224004,0.227204,0.448140,1.175131,0.637840,0.297238,0.524205,0.239352,0.373665,0.448146,0.636074,0.497337,0.235467,0.237620,0.362685,0.647145,0.800843,0.879639,1.052920,0.486257,0.620928,0.604825,0.669249,0.752518,0.654761,0.922859
3,HOBBIES_1_004_CA_1_validation,2.328879,0.992148,0.703719,1.372625,1.303904,1.455103,2.707075,1.576173,1.056397,0.813896,1.549485,1.533241,2.334356,2.396921,2.095440,1.261296,1.430184,1.355523,1.831998,2.042552,3.000276,1.790591,1.376936,1.708965,1.012837,1.357434,3.458492,3.126287
4,HOBBIES_1_005_CA_1_validation,1.124764,0.962933,1.157911,0.963144,1.649291,1.343001,1.259075,1.051243,1.370573,0.993080,0.947763,1.076977,1.069260,1.244801,1.121759,0.863347,1.201216,0.912491,1.007229,0.992169,1.503367,0.820870,0.816862,0.943335,1.194283,0.923853,1.188967,1.995593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.465499,0.403143,0.275166,0.663804,0.789565,0.621474,0.698561,0.410837,0.608729,0.307860,0.946598,0.930575,0.526679,0.813391,0.657703,0.627410,0.519883,0.633088,0.966022,0.529932,1.175308,0.298765,1.068147,0.595146,0.587710,0.739363,0.571244,0.937538
60976,FOODS_3_824_WI_3_evaluation,0.590313,0.477654,0.436830,0.587255,0.449119,0.698782,0.462273,0.509337,0.522048,0.408965,0.866013,0.402242,0.725149,0.477885,0.428285,0.555101,0.424512,0.689215,0.307395,0.478562,0.431488,0.303480,0.467018,0.425645,0.628929,0.304253,0.729342,0.341851
60977,FOODS_3_825_WI_3_evaluation,1.227312,0.281559,0.807025,0.400255,0.572666,1.118447,0.676378,1.306084,0.423004,0.949345,0.550833,0.519745,0.978226,0.844389,1.121399,0.861190,0.903350,1.060624,0.864625,0.842725,0.713519,0.821559,0.770557,1.218992,0.472456,0.571609,0.516877,0.921955
60978,FOODS_3_826_WI_3_evaluation,1.158328,1.644730,0.725141,1.411336,1.187850,0.852136,1.614389,1.009809,1.140077,1.040822,1.308934,1.320022,1.329711,1.332081,1.292687,1.048972,1.110637,1.290987,1.321319,1.065957,1.240711,1.489312,1.875940,1.604342,1.519513,1.295796,1.124396,1.172219


In [26]:
sub.to_csv("lgb_sub_V6.csv",index = False)

In [ ]:
pd.read_csv('submission.csv')

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.635264,0.665307,0.549572,0.790765,0.798116,1.242220,0.806613,0.621331,0.985524,0.688249,1.750373,0.735975,1.441405,1.580621,0.355633,0.827599,0.650038,0.938422,0.533167,1.521630,1.433755,0.831869,0.931035,0.810626,0.886726,1.101035,1.808653,0.741226
1,HOBBIES_1_002_CA_1_validation,0.376999,0.288994,0.363103,0.381820,0.232232,0.327193,0.290006,0.276124,0.343547,0.310717,0.349650,0.230558,0.289352,0.266013,0.262656,0.308625,0.347597,0.313840,0.220531,0.290091,0.278131,0.246310,0.153367,0.397939,0.246211,0.299183,0.445864,0.603483
2,HOBBIES_1_003_CA_1_validation,0.418780,0.478669,0.508481,0.496547,0.710814,0.860313,0.777815,0.349813,0.531672,0.489414,0.434220,0.413290,1.109061,0.576684,0.216356,0.252927,0.419898,0.575693,0.746556,1.308608,1.193171,0.368092,0.554398,0.500015,0.463380,0.907216,0.555819,1.407281
3,HOBBIES_1_004_CA_1_validation,2.307070,0.733208,0.887965,1.180000,1.684780,2.724603,3.457966,1.695340,0.799806,0.658726,1.608318,2.073488,2.518639,3.600225,2.211253,0.759219,1.016658,1.012015,2.038040,1.611736,3.633332,2.242869,0.989729,1.171726,0.908973,1.511999,2.024376,2.301502
4,HOBBIES_1_005_CA_1_validation,0.978023,0.826050,0.873653,1.224260,1.487269,1.258053,1.786572,0.866067,0.958617,0.985152,1.636083,1.165685,0.834632,1.875338,1.153875,1.147669,1.108804,1.130619,1.388660,0.863283,1.767650,1.160473,1.317741,0.981237,1.111697,1.301603,1.108424,1.627276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.307991,0.442061,0.568413,0.397280,0.615776,0.587419,0.960979,0.253106,0.499260,0.403834,0.604434,0.606350,0.459687,0.676348,0.418505,0.398572,0.310596,0.447556,0.626073,0.329978,0.700339,0.255816,0.678952,0.403750,0.376869,0.542999,0.313316,0.535191
60976,FOODS_3_824_WI_3_evaluation,0.383718,0.287527,0.241022,0.518830,0.254843,0.591806,0.257437,0.273208,0.333609,0.245650,0.714134,0.256479,0.630584,0.343458,0.358837,0.350766,0.282421,0.451720,0.163945,0.389621,0.341670,0.298383,0.360118,0.285289,0.414636,0.161167,0.411155,0.196606
60977,FOODS_3_825_WI_3_evaluation,1.193915,0.788892,1.041185,0.411093,0.751011,0.670638,0.769783,1.136382,0.258346,0.736198,0.367322,0.685137,0.618515,0.938525,0.741838,0.513635,0.836019,0.712328,0.468488,0.665763,0.551860,0.777720,0.503953,0.902870,0.298881,0.409184,0.425125,0.628896
60978,FOODS_3_826_WI_3_evaluation,1.471955,1.912015,0.818055,1.112489,1.112249,1.078456,1.285677,1.315327,1.401554,0.803920,0.984632,0.941785,1.044369,0.999166,0.990831,0.701703,0.703931,0.839125,0.844645,0.745097,0.891677,0.815621,1.089787,0.825554,1.096770,0.664142,0.665504,0.790551


In [27]:
!kaggle competitions submit -c m5-forecasting-accuracy -f lgb_sub_V6.csv -m "Much more optimized and changed some codes"

100% 32.7M/32.7M [00:11<00:00, 2.95MB/s]
Successfully submitted to M5 Forecasting - Accuracy